<a href="https://colab.research.google.com/github/HungChunLi/MacroFinalReport/blob/main/TWSE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso, LassoCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import warnings

In [2]:
!wget test.csv https://www.dropbox.com/scl/fi/wiepra10f739av4pwsyyl/Tse_06_20190102.csv?rlkey=gv9w5gt48l6lcertyxxtreetm&st=lknbwdzv&dl=0

--2024-05-04 17:20:11--  http://test.csv/
Resolving test.csv (test.csv)... failed: Name or service not known.
wget: unable to resolve host address ‘test.csv’
--2024-05-04 17:20:11--  https://www.dropbox.com/scl/fi/wiepra10f739av4pwsyyl/Tse_06_20190102.csv?rlkey=gv9w5gt48l6lcertyxxtreetm
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6020:18::a27d:4012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uccec246153207501926aa7c069e.dl.dropboxusercontent.com/cd/0/inline/CSTifQxljVyZ3jxi9lHdldhKXi1AygAP5udWB-kQVLEYjYkgMQlDAG6wNaTGiyVx7eDH-bqUQwMYUoj7lXnHlRApM3-F28qnUmYHpXc029QiF_NufvHIjVqJr3-DRc8fnBD5vKg8D9OpYygu0gTPYmSR/file# [following]
--2024-05-04 17:20:12--  https://uccec246153207501926aa7c069e.dl.dropboxusercontent.com/cd/0/inline/CSTifQxljVyZ3jxi9lHdldhKXi1AygAP5udWB-kQVLEYjYkgMQlDAG6wNaTGiyVx7eDH-bqUQwMYUoj7lXnHlRApM3-F28qnUmYHpXc029QiF_NufvHIjVqJr3-DRc8fnBD5vKg8D

In [5]:
PATH = '/content/Tse_06_20190102.csv?rlkey=gv9w5gt48l6lcertyxxtreetm'
TWSE_trade = pd.read_csv(PATH, encoding='cp950')

<ipython-input-5-2ee6ac336e77>:2: DtypeWarning: Columns (1,2,3,4,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43) have mixed types. Specify dtype option on import or set low_memory=False.
  NYSE_trade = pd.read_csv(PATH, encoding='cp950')


In [6]:
NYSE_trade.head()

,zdate,b_mkt,b_format,b_version,d_serial,stk_id7,stk_time,d_deal,d_buy_count,d_sell_count,...,sp_1,sv_1,sp_2,sv_2,sp_3,sv_3,sp_4,sv_4,sp_5,sv_5
0,日期,市場別,格式碼,格式版本,傳輸序號,股票代號,撮合時間,成交註記,買進價量檔數,賣出價量檔數,...,最佳一檔賣出價格,最佳一檔賣出數量,最佳二檔賣出價格,最佳二檔賣出數量,最佳三檔賣出價格,最佳三檔賣出數量,最佳四檔賣出價格,最佳四檔賣出數量,最佳五檔賣出價格,最佳五檔賣出數量
1,2019-01-02,01,06,03,1,00637L,2019-01-02 08:30:00.295813,0,0,2,...,11.42,2,11.63,10,0.00,0,0.00,0,0.00,0
2,2019-01-02,01,06,03,2,2455,2019-01-02 08:30:00.295813,0,0,1,...,71.00,15,0.00,0,0.00,0,0.00,0,0.00,0
3,2019-01-02,01,06,03,3,00672L,2019-01-02 08:30:00.400806,0,1,3,...,13.66,1,13.98,1,14.28,1,0.00,0,0.00,0
4,2019-01-02,01,06,03,4,3037,2019-01-02 08:30:00.400806,0,0,2,...,23.50,6,24.50,5,0.00,0,0.00,0,0.00,0


In [7]:
NYSE_trade.tail()

,zdate,b_mkt,b_format,b_version,d_serial,stk_id7,stk_time,d_deal,d_buy_count,d_sell_count,...,sp_1,sv_1,sp_2,sv_2,sp_3,sv_3,sp_4,sv_4,sp_5,sv_5
768036,2019-01-02,1,6,3,768036,1720,2019-01-02 13:30:00,1,5,5,...,31.4,7,31.5,1,31.6,2,31.7,1,31.75,4
768037,2019-01-02,1,6,3,768037,1810,2019-01-02 13:30:00,1,5,5,...,7.83,19,7.85,6,7.86,1,7.87,1,7.88,1
768038,2019-01-02,1,6,3,768038,2516,2019-01-02 13:30:00,1,5,5,...,6.36,8,6.37,5,6.38,4,6.4,6,6.43,3
768039,2019-01-02,1,6,3,768039,1538,2019-01-02 13:33:00,1,5,5,...,5.17,1,5.18,1,5.19,2,5.24,1,5.25,3
768040,2019-01-02,1,6,3,768040,000000,2019-01-06 04:40:39.999999,0,0,0,...,0.0,0,0.0,0,0.0,0,0.0,0,0.0,0


In [9]:
print(NYSE_trade.shape)
print(NYSE_trade.columns)

(768041, 44)
Index(['zdate', 'b_mkt', 'b_format', 'b_version', 'd_serial', 'stk_id7',
       'stk_time', 'd_deal', 'd_buy_count', 'd_sell_count', 'd_five',
       'd_deal_mark', 'd_buy_mark', 'd_sell_mark', 'd_instant_mark',
       'd_trail_mark', 'd_trail_open_delay', 'd_trail_close_delay',
       'd_trade_mode', 'd_open_mark', 'd_close_mark', 'vol_d', 'zclosed',
       'vol_n', 'bp_1', 'bv_1', 'bp_2', 'bv_2', 'bp_3', 'bv_3', 'bp_4', 'bv_4',
       'bp_5', 'bv_5', 'sp_1', 'sv_1', 'sp_2', 'sv_2', 'sp_3', 'sv_3', 'sp_4',
       'sv_4', 'sp_5', 'sv_5'],
      dtype='object')
